In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
import numpy as np
from site_analysis import Atom, Analysis, PolyhedralSite
from site_analysis.tools import get_nearest_neighbour_indices
from collections import Counter
import tqdm
import matplotlib.pyplot as plt
import yaml

import sys
sys.path.insert(0, "../../scripts/")
from utils import flatten_list, get_structures, indices_by_species

from scipy.stats import gaussian_kde
from scipy.ndimage.filters import gaussian_filter1d

In [55]:
directory = '../../data'
structure = Poscar.from_file(f'{directory}/reference_structures/Li6PS5I_alltet_sites.POSCAR.vasp').structure
from pymatgen import Structure
lattice = structure.lattice
t0 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['P'], coords=[[0.5, 0.0, 0.0]])
t1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.9, 0.9, 0.6]])
t2 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.23, 0.93, 0.07]])
t3 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.25, 0.25, 0.25]])
t4 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.15, 0.15, 0.15]])
t5 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.0, 0.183, 0.183]])
tet_sites = [t0, t1, t2, t3, t4, t5]

t_all = Structure.from_sites(flatten_list([(t*[2,2,2]).sites for t in tet_sites]))

In [56]:
s_sites = Structure.from_spacegroup(sg='F-43m', 
                                    lattice=lattice,
                                    species=['S', 'S', 'S'], 
                                    coords=[[0.0, 0.0, 0.0],
                                            [0.75, 0.25, 0.25],
                                            [0.11824, 0.11824, 0.38176]])*[2,2,2]

In [57]:
s_sites.to(filename='test.cif')

In [43]:
md_ref_structure = Xdatcar(f'{directory}/Li6PS5Cl/50p/run1/inherent_xdatcar.gz').structures[0]

In [44]:
def map_to_nearest_site(input_structure, species_strings, ref_structure):
    site_indices = [i for i, s in enumerate(input_structure) if s.species_string in species_strings]
    sites = [input_structure[i] for i in site_indices]
    frac_coords = [input_structure[i].frac_coords for i in site_indices]
    latt = ref_structure.lattice
    dr = latt.get_all_distances( frac_coords, ref_structure.frac_coords )
    for s, r in zip(sites, dr):
        index = np.argmin(r)
        s.coords = ref_structure[index].coords
    return input_structure

        
map_to_nearest_site(md_ref_structure, ['Cl', 'S'], s_sites)

Structure Summary
Lattice
    abc : 20.312311001202943 20.31233900091543 20.31242400034112
 angles : 90.00018617127729 90.00063748668664 89.99892811843388
 volume : 8380.714126120198
      A : 20.312311 0.00019 -0.000113
      B : 0.00019 20.312339 -3.3e-05
      C : -0.000113 -3.3e-05 20.312424
PeriodicSite: Li (0.4173, 2.9610, 6.3509) [0.0205, 0.1458, 0.3127]
PeriodicSite: Li (0.7857, 3.1540, 16.1484) [0.0387, 0.1553, 0.7950]
PeriodicSite: Li (2.9464, 13.2861, 10.2245) [0.1450, 0.6541, 0.5034]
PeriodicSite: Li (1.7500, 14.9706, 17.2819) [0.0862, 0.7370, 0.8508]
PeriodicSite: Li (14.5972, 0.8147, 7.5381) [0.7186, 0.0401, 0.3711]
PeriodicSite: Li (12.1687, 4.3278, 16.0891) [0.5991, 0.2131, 0.7921]
PeriodicSite: Li (15.4574, 12.1756, 3.4617) [0.7610, 0.5994, 0.1704]
PeriodicSite: Li (14.8291, 10.7601, 17.6003) [0.7301, 0.5297, 0.8665]
PeriodicSite: Li (3.0484, 4.7270, 8.4698) [0.1501, 0.2327, 0.4170]
PeriodicSite: Li (3.3857, 0.2486, 17.3226) [0.1667, 0.0122, 0.8528]
PeriodicSite: Li (5

In [41]:
sites

[PeriodicSite: Cl (6.2698, 11.3405, 8.9422) [0.3087, 0.5583, 0.4402],
 PeriodicSite: Cl (1.1991, 14.0129, 1.1991) [0.0590, 0.6899, 0.0590],
 PeriodicSite: Cl (16.4111, 16.4111, 14.0129) [0.8079, 0.8079, 0.6899],
 PeriodicSite: Cl (14.0129, 8.9422, 19.0836) [0.6899, 0.4402, 0.9395],
 PeriodicSite: Cl (8.9422, 3.8716, 19.0836) [0.4402, 0.1906, 0.9395],
 PeriodicSite: Cl (6.2698, 6.2698, 3.8716) [0.3087, 0.3087, 0.1906],
 PeriodicSite: Cl (14.0129, 8.9422, 8.9422) [0.6899, 0.4402, 0.4402],
 PeriodicSite: Cl (1.1991, 8.9422, 6.2698) [0.0590, 0.4402, 0.3087],
 PeriodicSite: Cl (6.2698, 6.2698, 14.0129) [0.3087, 0.3087, 0.6899],
 PeriodicSite: Cl (16.4111, 14.0129, 16.4111) [0.8079, 0.6899, 0.8079],
 PeriodicSite: Cl (16.4111, 1.1991, 19.0836) [0.8079, 0.0590, 0.9395],
 PeriodicSite: Cl (1.1991, 19.0836, 6.2698) [0.0590, 0.9395, 0.3087],
 PeriodicSite: Cl (11.3405, 6.2698, 19.0836) [0.5583, 0.3087, 0.9395],
 PeriodicSite: Cl (16.4111, 8.9422, 1.1991) [0.8079, 0.4402, 0.0590],
 PeriodicSite: 

TypeError: list indices must be integers or slices, not list

In [29]:
md_ref_structure

Structure Summary
Lattice
    abc : 20.312311001202943 20.31233900091543 20.31242400034112
 angles : 90.00018617127729 90.00063748668664 89.99892811843388
 volume : 8380.714126120198
      A : 20.312311 0.00019 -0.000113
      B : 0.00019 20.312339 -3.3e-05
      C : -0.000113 -3.3e-05 20.312424
PeriodicSite: Li (0.4173, 2.9610, 6.3509) [0.0205, 0.1458, 0.3127]
PeriodicSite: Li (0.7857, 3.1540, 16.1484) [0.0387, 0.1553, 0.7950]
PeriodicSite: Li (2.9464, 13.2861, 10.2245) [0.1450, 0.6541, 0.5034]
PeriodicSite: Li (1.7500, 14.9706, 17.2819) [0.0862, 0.7370, 0.8508]
PeriodicSite: Li (14.5972, 0.8147, 7.5381) [0.7186, 0.0401, 0.3711]
PeriodicSite: Li (12.1687, 4.3278, 16.0891) [0.5991, 0.2131, 0.7921]
PeriodicSite: Li (15.4574, 12.1756, 3.4617) [0.7610, 0.5994, 0.1704]
PeriodicSite: Li (14.8291, 10.7601, 17.6003) [0.7301, 0.5297, 0.8665]
PeriodicSite: Li (3.0484, 4.7270, 8.4698) [0.1501, 0.2327, 0.4170]
PeriodicSite: Li (3.3857, 0.2486, 17.3226) [0.1667, 0.0122, 0.8528]
PeriodicSite: Li (5

In [30]:
s = md_ref_structure[0]

In [31]:
s.frac_coords=[0,0,0]

In [32]:
md_ref_structure

Structure Summary
Lattice
    abc : 20.312311001202943 20.31233900091543 20.31242400034112
 angles : 90.00018617127729 90.00063748668664 89.99892811843388
 volume : 8380.714126120198
      A : 20.312311 0.00019 -0.000113
      B : 0.00019 20.312339 -3.3e-05
      C : -0.000113 -3.3e-05 20.312424
PeriodicSite: Li (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Li (0.7857, 3.1540, 16.1484) [0.0387, 0.1553, 0.7950]
PeriodicSite: Li (2.9464, 13.2861, 10.2245) [0.1450, 0.6541, 0.5034]
PeriodicSite: Li (1.7500, 14.9706, 17.2819) [0.0862, 0.7370, 0.8508]
PeriodicSite: Li (14.5972, 0.8147, 7.5381) [0.7186, 0.0401, 0.3711]
PeriodicSite: Li (12.1687, 4.3278, 16.0891) [0.5991, 0.2131, 0.7921]
PeriodicSite: Li (15.4574, 12.1756, 3.4617) [0.7610, 0.5994, 0.1704]
PeriodicSite: Li (14.8291, 10.7601, 17.6003) [0.7301, 0.5297, 0.8665]
PeriodicSite: Li (3.0484, 4.7270, 8.4698) [0.1501, 0.2327, 0.4170]
PeriodicSite: Li (3.3857, 0.2486, 17.3226) [0.1667, 0.0122, 0.8528]
PeriodicSite: Li (5